#### https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

train_df = pd.read_csv('data/train.csv', parse_dates=["date"])
test_df = pd.read_csv("data/test.csv", parse_dates=["date"])
stores_df = pd.read_csv("data/stores.csv")
oil_df = pd.read_csv("data/oil.csv", parse_dates=["date"])
holidays_df = pd.read_csv("data/holidays_events.csv", parse_dates=["date"])
transactions_df = pd.read_csv("data/transactions.csv", parse_dates=["date"])

In [5]:
# Display basic info
print(train_df.head())
print(train_df.info())

   id       date  store_nbr      family  sales  onpromotion
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0
1   1 2013-01-01          1   BABY CARE    0.0            0
2   2 2013-01-01          1      BEAUTY    0.0            0
3   3 2013-01-01          1   BEVERAGES    0.0            0
4   4 2013-01-01          1       BOOKS    0.0            0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB
None


In [14]:
train_df['family'].nunique()
# 'nunique()': Use this method when you only need the count of unique values.

33

In [16]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [19]:
# Merge transactions into train data
train_df = train_df.merge(transactions_df, on=["date", "store_nbr"], how="left")

In [35]:
train_df['transactions'].count()

np.int64(2755104)

In [32]:
train_df['transactions'].isna().sum()

np.int64(245784)

In [37]:
len(train_df['transactions'])

3000888

In [33]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,NaN


In [38]:
# Fill missing transaction values (if any) with the median
train_df["transactions"].fillna(train_df["transactions"].median(), inplace=True)

In [42]:
train_df.head(20)

,id,date,store_nbr,family,sales,onpromotion,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1393.0
1,1,2013-01-01,1,BABY CARE,0.0,0,1393.0
2,2,2013-01-01,1,BEAUTY,0.0,0,1393.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,1393.0
4,4,2013-01-01,1,BOOKS,0.0,0,1393.0
5,5,2013-01-01,1,BREAD/BAKERY,0.0,0,1393.0
6,6,2013-01-01,1,CELEBRATION,0.0,0,1393.0
7,7,2013-01-01,1,CLEANING,0.0,0,1393.0
8,8,2013-01-01,1,DAIRY,0.0,0,1393.0
9,9,2013-01-01,1,DELI,0.0,0,1393.0
